In [1]:
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
from bs4 import BeautifulSoup
import math
import asyncio
import aiohttp
import re

In [2]:
disable_warnings(InsecureRequestWarning)

In [3]:
def __checkForEndOfPages(html, page):
    """checks if the underlined pagenumber in the @html string is equivilant to the number given as @page

    Args:
        html (str): html-string containing one page
        page (int): page number

    Returns:
        bool: if the page numbers are the same -> False (not end of Pages). If page numbers are not the same -> True
    """
    return int(BeautifulSoup(html, 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0]) != page

def __deleteRedundantPages(res, div):
    # TODO: improve work -> better use inbuild search function
    """delete all pages from the end, that are redundant

    Args:
        res (list of string): list that contains all pages
        div (int): amount of pages per iteration

    Returns:
        list of string: list that contains all pages (now without redundant pages at the end)
    """
    if div == 1:
        res = res[:-1]
    else:
        buf = res[-div:]
        res = res[:-div]
        
        parse_buffer1 = int(BeautifulSoup(buf[-1], 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0])
        for i in reversed(range(1, div)):
            parse_buffer2 = int(BeautifulSoup(buf[i-1], 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0])
            if parse_buffer1 == parse_buffer2:
                buf.pop()
                if i == 1 and len(res) > 0:
                    if int(BeautifulSoup(buf[i-1], 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0]) == int(BeautifulSoup(res[-1], 'html.parser').find('div', class_='simpleRoundedBoxTitleGreyTall').find('div', class_='pagingcontrols').find('a', style='text-decoration:underline;').contents[0]):
                        buf.pop()
            else:
                break
            parse_buffer1 = parse_buffer2
        res += buf
    return res

async def fetch(s, url):
    async with s.get(f'https://gatherer.wizards.com/Pages/Search/Default.aspx?sort=cn+&page={url}&name=%20[]') as r:
        if r.status != 200:
            r.raise_for_status()
        return await r.text()

async def fetch_all(s, pageNum, offset, div):
    tasks = []
    res = []
    end = math.ceil(pageNum/div)
    if pageNum != 0:
        for i in range(int(offset/div), end + int(offset/div)):
            for i2 in range(i*div, (i + 1)*div):
                task = asyncio.create_task(fetch(s, i2))
                tasks.append(task)
            res = await asyncio.gather(*tasks)
            print(str(i*div) + "..." + str((i+1)*div-1))
            if __checkForEndOfPages(res[-1], (i + 1)*div):
                res = __deleteRedundantPages(res, div)
                break  
        if pageNum < len(res):
            res = res[:pageNum]
    else:
        i = int(offset/div)
        while 1:
            for i2 in range(i*div, (i + 1)*div):
                task = asyncio.create_task(fetch(s, i2))
                tasks.append(task)
            res = await asyncio.gather(*tasks)
            print(str(i*div) + "..." + str((i+1)*div-1))
            if __checkForEndOfPages(res[-1], (i + 1)*div):
                res = __deleteRedundantPages(res, div)
                break
            i += 1
    return res

async def main(pageNum = 0):
    div = 20        # integer
    offset = 200      # integer - sollte Vielfaches von div sein - bei div = 10 -> offset = 10 oder = 20 oder ... oder = 250
                    # wird als offset = 241 gewählt, so wird intern daraus 240 gemacht, genauso bei offset = 242 oder ... oder 249 ... usw.
    if div < 1:
        print("Div muss >= 1")
    else:
        async with aiohttp.ClientSession() as session:
            htmls = await fetch_all(session, pageNum, offset, div)
            #print(htmls)
            return htmls

#asyncio.run(main())
if __name__ == "__main__":
    htmls = await main(0)

200...219
220...239
240...259


In [4]:
len(htmls)

57

In [19]:
def getImageAndID(card):
    data = card.find('td', class_='leftCol').find('a').find('img')['src'][5:]
    return "https://gatherer.wizards.com" + data, re.search("multiverseid=(.*)&", data).group(1)

def getName(card):
    return card.find('td', class_='middleCol').find('div', class_='cardInfo').find('span', class_='cardTitle').find('a').contents[0]

def getType(card):
    return re.sub(' +', ' ', re.sub(r'\r\n', "", str(card.find('td', class_='middleCol').find('div', class_='cardInfo').find('span', class_='typeLine').contents[0]))).strip().replace("—", "-")

In [20]:
class csvCreator():
    
    def __init__(this):
        this.__csvCols = []
        this.__csvConstruct = []
        this.__csv = ''
    
    def defineCols(this, *params):
        this.__csvCols = []
        for param in params:
            this.__csvCols.append(param)
    
    def addCard(this, *params):
        this.__csvConstruct.append([])
        for param in params:
            if param[0] == "int" or param[0] == "float":
                this.__csvConstruct[-1].append(param[1])
            else:
                this.__csvConstruct[-1].append("\"" + str(param[1]) + "\"")
    
    def getCsv(this):
        this.__csv = this.__createRow(this.__csvCols)
        for row in this.__csvConstruct:
            this.__csv += this.__createRow(row)
        return this.__csv
    
    def __createRow(this, row):
        buf = ""
        for valorem in row:
            buf += str(valorem) + ","
        return buf[:-1] + "\n"
        

In [21]:
csv = csvCreator()
csv.defineCols("ID", "Image-ID", "Name", "Type")

for html in htmls:
    results = BeautifulSoup(html, 'html.parser')
    cards = results.find_all('tr', class_=['cardItem evenItem', 'cardItem oddItem'])
    #print(i)
    for card in cards:
        img, id = getImageAndID(card)
        csv.addCard(("int", id), ("str", img), ("str", getName(card)), ("str", getType(card)))
    
f = open("Lukas.csv", "w")
f.write(csv.getCsv())
f.close()